# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [17]:

# import libraries
import time
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

from pyspark.sql import Window, SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, DateType, DoubleType

from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler, MinMaxScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier



In [18]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Spark Project") \
    .getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [ ]:
file_path = "mini_sparkify_event_data.json"
df = spark.read.json(file_path)
df.head(1)

In [ ]:
df.printSchema()

In [ ]:
df.count()

In [ ]:
df.select(['userId']).distinct().count()

In [ ]:
# Get unique values of column "page"
unique_page_values = df.select("page").distinct()

# Show the unique values
unique_page_values.show()

In [ ]:
df.select([F.count(F.when((F.col(c).isNull()) | (F.col(c) == ""), c)).alias(c) for c in df.columns]).show()

In [ ]:
#filter out invalid users
valid_user_df = df.filter((F.col("userId").isNotNull()) & (F.col("userId") != ""))
valid_user_df.show(2)

In [ ]:
valid_user_df.count()

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
# Define Churn
churn_df = valid_user_df.withColumn(
            "isChurn",
            F.when(F.col("page").isin(["Cancellation Confirmation", "Downgrade"]), 1).otherwise(0))

churn_df.show(1)


In [ ]:
# Extract relevant columns and remove duplicates
level_df = churn_df.select("userId", "level", "isChurn").distinct()

# Plot the number of paid and free users
ax = sns.countplot(x="level", data=level_df.toPandas())
plt.title("Number of paid and free users")
plt.show()

In [ ]:
# Extract relevant columns and remove duplicates
gender_df = churn_df.select("userId", "gender", "isChurn").distinct()

# Plot gender distribution
ax = sns.countplot(x="gender", data=gender_df.toPandas())
plt.title("Gender distribution")
plt.show()


In [ ]:

# Plot gender distribution by user status
churn_by_gender_df = gender_df.groupBy("isChurn", "gender").count().sort("isChurn").toPandas()
ax = sns.barplot(data=churn_by_gender_df, x="isChurn", y="count", hue="gender")
plt.title("Gender Distribution by User Status", fontweight='bold')
plt.xlabel("User Status")
plt.ylabel("Amount of Users")
plt.xticks(np.arange(2), ["Active", "Cancelled"])
plt.legend(title="Gender")
plt.show()



In [ ]:
# Plot level distribution by user status
churn_by_level_df = level_df.groupBy("isChurn", "level").count().sort("isChurn").toPandas()
ax = sns.barplot(data=churn_by_level_df, x="isChurn", y="count", hue="level")
plt.title("Distribution Level by User Status", fontweight='bold')
plt.xlabel("User Status")
plt.ylabel("Amount of Users")
plt.xticks(np.arange(2), ["Active", "Cancelled"])
plt.legend(title="Level")
plt.show()

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
df_by_user = churn_df.groupBy('userId').agg(
    F.first('gender').alias('gender'),
    F.first('level').alias('level'),
    F.avg('length').alias('avg_song_length'),
    F.countDistinct('artist').alias('total_artist'),
    F.countDistinct('song').alias('total_song'),
    F.countDistinct('sessionId').alias('total_session'),
    F.count(F.when(F.col('page') == 'Thumbs Up', True)).alias('total_thumb_up'),
    F.count(F.when(F.col('page') == 'Thumbs Down', True)).alias('total_thumb_down'),
    F.count(F.when(F.col('page') == 'Add Friend', True)).alias('total_add_friend'),
    F.count(F.when(F.col('page') == 'Add to Playlist', True)).alias('total_add_playlist'),
    F.sum('isChurn').alias('totalChurn')
)
df_by_user = df_by_user.withColumn(
            "isChurn",
            F.when(F.col("totalChurn")>=1, 1).otherwise(0))

df_by_user = df_by_user.withColumn(
            "Male",
            F.when(F.col("gender")=='M', 1).otherwise(0))

df_by_user = df_by_user.withColumn(
            "paid",
            F.when(F.col("level")=='paid', 1).otherwise(0))

df_by_user = df_by_user.drop('gender', 'level', 'totalChurn')

df_by_user.show(1)


In [ ]:
df_by_user_vis = df_by_user.toPandas()

fig = plt.figure(figsize=(30, 25))
ax = fig.gca()
h = df_by_user_vis.hist(ax=ax)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
# Split the dataset into training and testing sets
(training_data, test_data) = df_by_user.randomSplit([0.7, 0.3], seed=13)

In [ ]:
feature_columns = ['avg_song_length', 'total_artist', 'total_song', 'total_session', \
                   'total_thumb_up', 'total_thumb_down', 'total_add_friend', 'total_add_playlist',\
                    'Male', 'paid']

In [ ]:
def build_model(classifier, param):
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
    pipeline = Pipeline(stages=[assembler, scaler, classifier])
    model = CrossValidator(
        estimator=pipeline,
        estimatorParamMaps=param,
        evaluator=MulticlassClassificationEvaluator(labelCol='isChurn', metricName='f1'),
        numFolds=5,
    )
    return model

In [ ]:
def evaluate_model(pred):
    evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="isChurn")
    f1_score = evaluator.evaluate(pred, {evaluator.metricName: "f1"})
    print('F1 score: {}'.format(f1_score))

In [ ]:
#### Logistic Regression

In [ ]:
logisticRegression = LogisticRegression(featuresCol="scaled_features", labelCol="isChurn")
parameterLogisticRegression = ParamGridBuilder().build()
model = build_model(logisticRegression, parameterLogisticRegression)

In [ ]:
start_time = time.time()
LogisticRegressModel = model.fit(training_data)
end_time = time.time()
print("Train time:", end_time - start_time)

In [ ]:
LogisticRegressionPrediction = LogisticRegressModel.transform(test_data)

In [ ]:
evaluate_model(LogisticRegressionPrediction)

In [ ]:
#### RandomForest

In [ ]:
RandomForest = RandomForestClassifier(featuresCol="scaled_features", labelCol="isChurn")
RandomForestParameter = ParamGridBuilder().build()
model1 = build_model(RandomForest, RandomForestParameter)

In [ ]:
start_time = time.time()
RandomForestModel = model1.fit(training_data)
print("Train time:", time.time() - start_time)

In [ ]:
RandomForestPrediction = RandomForestModel.transform(test_data)

In [ ]:
evaluate_model(RandomForestPrediction)

In [ ]:
#### GBT Classifier

In [ ]:
GBT = GBTClassifier(featuresCol="scaled_features", labelCol="isChurn")
GBTParameter = ParamGridBuilder().build()
model2 = build_model(GBT, GBTParameter)

In [ ]:
start_time = time.time()
GBTModel = model2.fit(training_data)
end_time = time.time()
print("Train time:", end_time - start_time)

In [ ]:
GBTModelPrediction = GBTModel.transform(test)

In [ ]:
evaluate_model(GBTModelPrediction)

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.